# 创建engine

In [1]:
from sqlalchemy import create_engine

# echo默认False不打印SQL语句详情，Ture表示打印。
engine = create_engine("sqlite:///tutorial.db", echo=False)

# 定义表结构

In [2]:
from sqlalchemy import Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import declarative_base

Base = declarative_base()


class User(Base):
    # 指定本类映射到users表
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(20))
    fullname = Column(String(32))
    password = Column(String(32))

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
                   self.name, self.fullname, self.password)

# 创建表


In [3]:
User.__table__
# checkfirst表示创建表前先检查该表是否存在,已存在则不再创建(默认True)
# Base.metadata.create_all(engine, checkfirst=True)
User.__table__.create(engine, checkfirst=False)

# 创建 session


In [4]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

## 增


In [5]:
ed_user = User(name="ed", fullname="Ed Jones", password="edspassword")

# 将该实例插入到users表
session.add(ed_user)

# 一次插入多条记录形式
session.add_all(
    [
        User(name="wendy", fullname="Wendy Williams", password="foobar"),
        User(name="mary", fullname="Mary Contrary", password="xxg527"),
        User(name="fred", fullname="Fred Flinstone", password="blah"),
    ]
)
# 当前更改只是在session中，需要使用commit确认更改才会写入数据库
session.commit()

## 查


### 单条


In [6]:
our_user = session.query(User).filter_by(name="ed").first()
print(our_user)

<User(name='ed', fullname='Ed Jones', password='edspassword')>


### 比较插入前的记录


In [7]:
# 比较ed_user与查询到的our_user是否为同一条记录
ed_user is our_user

True

### 查询全部


In [8]:
all_user = session.query(User.name).all()
print(all_user)

[('ed',), ('wendy',), ('mary',), ('fred',)]


## 改


In [9]:
mod_user = session.query(User).filter_by(name="ed").first()
print(mod_user)
mod_user.password = "modify_passwd"
# 确认修改
session.commit()
print(session.query(User).filter_by(name="ed").first())

<User(name='ed', fullname='Ed Jones', password='edspassword')>
<User(name='ed', fullname='Ed Jones', password='modify_passwd')>


## 删


In [10]:
del_user = session.query(User).filter_by(name="ed").first()
session.delete(del_user)
session.commit()
for user in session.query(User):
    print(user)

<User(name='wendy', fullname='Wendy Williams', password='foobar')>
<User(name='mary', fullname='Mary Contrary', password='xxg527')>
<User(name='fred', fullname='Fred Flinstone', password='blah')>
